In [4]:
import itertools
from bidict import bidict
from collections import defaultdict
import time 

WE = defaultdict(lambda: None)

m=defaultdict(lambda: None)

def Wedderburn(n):
    if WE[n] == None: 
        if n == 1:
            WE[n] = 1
        elif n % 2 == 1:
            s = sum([Wedderburn(i) * Wedderburn(n-i) for i in range(1, (n+1)/2)])
            WE[n] = s
        else:
            s = sum([Wedderburn(i) * Wedderburn(n-i) for i in range(1, n/2)])
            s += (Wedderburn(n/2) * (Wedderburn(n/2) + 1)) / 2
            WE[n] = s
    return WE[n]

isom_classes= defaultdict(bidict)

isom_classes[1] = bidict({0 : ((1,0),(0,0))})
isom_classes[2] = bidict({0 : ((1,0),(1,0))})
isom_classes[3] = bidict({0 : ((2,0),(1,0))})

import itertools

#inputs list/set of pairs and outputs a set of ordered pairs
def unordered_pairs(lst1, lst2):
    if len(lst1) == 0 or len(lst1) == 0:
        return Set({})
    return Set((y, x) if x <= y else (x, y) for x, y in set(itertools.product(lst1, lst2)))

def list_BT(n):
    for i in range(4,n+1):
        if len(isom_classes[i]) == 0:
            counter = 0
            for ell in Partitions(i, length=2):
                if ell[0] == ell[1]: 
                    for x in unordered_pairs(isom_classes[ell[0]] , isom_classes[ell[0]]):
                        isom_classes[i][counter] = ((ell[0],x[0]),(ell[0],x[1]))
                        counter+=1
                else: 
                    for x in itertools.product(isom_classes[ell[0]] , isom_classes[ell[1]]):
                        isom_classes[i][counter] = ((ell[0],x[0]),(ell[1],x[1]))
                        counter+=1 
    return isom_classes[n]   


#inputs tree_0 = (n_0,s_0) and tree_1 = (n_1,s_1) and outputs the tree (n,s) such that tree_0 + tree_1 = (n,s)
def binary_tree(tree_0,tree_1):
    n_0 = tree_0[0]
    n_1 = tree_1[0]
    n = n_0+n_1
    if n_0 == 0: 
        return tree_1
    if n_1 == 0: 
        return tree_0
    elif tree_0 <= tree_1:
        return (n,list_BT(n).inverse[(tree_1,tree_0)])
    else:
        return (n,list_BT(n).inverse[(tree_0,tree_1)])

#intakes pair = (n,s) and an integer k and outputs the deck as a set
def deck(pair, k):
    if m[(pair,k)] == None:
        n = pair[0]
        s = pair[1]
        if k > n: 
            m[(pair,k)] = set()
        if k == 0: 
            m[(pair,k)] = set([(0, 0)])
        if k == n:
            m[(pair,k)] = set([pair])
        if n > k > 0:
            deck_set = set()
            root_split = list_BT(n)[s]
            left_tree = root_split[0]
            right_tree = root_split[1]
            r = min(k,left_tree[0])
            for i in [x for x in range(r+1) if k-x <= right_tree[0]]:
                for element in unordered_pairs( deck( left_tree, i) , deck( right_tree,k-i) ):
                    card = binary_tree(element[0],element[1])
                    deck_set.add(card)
            m[(pair,k)] = deck_set
    return m[(pair,k)]

#inputs a pair = (n,s) and outputs the s'th binary tree on n leaves       
def convert_back(pair):
    T_1 = BinaryTree([])
    n = pair[0]
    s = pair[1]
    if n == 1:
        return T_1
    if n > 1:
        pair = list_BT(n)[s]
        T_left = convert_back(pair[0])
        T_right = convert_back(pair[1])
        return BinaryTree([T_left,T_right])

#intakes pair = (n,s) and draws it in ascii_art (must build every )
def draw(pair):
    print(ascii_art(convert_back(pair)))    

#input: k
#output: list of universal trees for k as tuples
def universal(k): 
    with open(f'universal_trees_{k}.txt', 'r') as file:
        lst= []
        for i, line in enumerate(file):
            lst.append(tuple(eval(line)))
        return lst  

#input: k = leaf-number, U = candidate for universal tree for k
#output: if U is universal, print T and write it on universal_trees_k as a new line
def universal_update(U,k): 
    if len(deck(U,k)) == Wedderburn(k):
        # print(U)
        # draw(U)
        with open(f'universal_trees_{k}.txt', 'a') as file:
            file.write(str(U) + "\n")

def add_one(T):
    S = universal(1)[0]
    return binary_tree(T,S)

def add_two(T):
    S = universal(2)[0]
    return binary_tree(T,S)

def add_three(T): 
    S = universal(3)[0]
    return binary_tree(T,S)

T_1 = universal(1)[0]
T_3 = universal(3)[0]

cache = defaultdict(lambda: None)

def recursive_bt_seq(k):
    if k == 0: 
        return T_1
    elif k % 2 == 0: 
        if cache[k] == None: 
            bt = binary_tree(recursive_bt_seq(k-1),T_3)
            cache[k] = bt
        return cache[k]
    else:
        if cache[k] == None:
            bt = binary_tree(recursive_bt_seq(k-1),T_1)
            cache[k] = bt
        return cache[k]

limit = 8

#for each k, we calculate the number of non-isomorphic induced subtrees of X_k and Y_k
for k in range(limit):
    T = recursive_bt_seq(k)
    # print(T)
    draw(T)
    print("total number of non-isomorphic induced subtrees:", sum([ len(deck(T,ell)) for ell in range(1,T[0]+1) ]) )

o
total number of non-isomorphic induced subtrees: 1
  o
 / \
o   o
total number of non-isomorphic induced subtrees: 2
       __o___
      /      \
    _o_       o
   /   \     / \
  o     o   o   o
 / \        
o   o       
total number of non-isomorphic induced subtrees: 6
           ___o____
          /        \
       __o___       o
      /      \      
    _o_       o     
   /   \     / \    
  o     o   o   o   
 / \                
o   o               
total number of non-isomorphic induced subtrees: 9
                _____o______
               /            \
           ___o____         _o_
          /        \       /   \
       __o___       o     o     o
      /      \           / \    
    _o_       o         o   o   
   /   \     / \        
  o     o   o   o       
 / \                    
o   o                   
total number of non-isomorphic induced subtrees: 25
                       _____o______
                      /            \
                _____o______       